# Methodological Foundation of a Numerical Taxonomy of Urban Form

## Reproducible Python code to generate taxonomy

Complete morphometrics assessment from input data to taxonomy.

Input data:
 - building footprints
 - street network
 
It is assumed that input data are cleaned to a required standard and stored in a single GeoPackage `files/geometry.gpkg` with two layers named `buildings` and `streets`. `buildings` area Polygons, whilst `streets` are LineStrings.

Buildings data contain two attribute columns:

- `sdbHei` is building height in meters.
- `floor_area` is gross floor area (area * number of floors).

This notebook requires `momepy` 0.3 or newer. The reproducible computational environment can be created using Docker container `darribas/gds_py:5.0`.

The same code has been used to analyse both cases.

### Generate additional morphometric elements

Before we can start morhometrics we have to generate additional elements - tessellation and tessellation based blocks.

First we import all required libraries.

In [ ]:
import geopandas as gpd
import momepy as mm
import libpysal
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import mapclassify

from inequality.theil import Theil
from tqdm import tqdm
from momepy import limit_range
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture
from scipy.cluster import hierarchy

We load buildings and create unique ID.

In [ ]:
path = "files/geometry.gpkg"
layer = "buildings"

In [ ]:
buildings = gpd.read_file(path, layer=layer)
buildings["uID"] = range(len(buildings))

In [ ]:
buildings.plot()

#### Morphological tessellation

Check input for tessellation. If the input data is clean, the check will result in zeros.

In [ ]:
check = mm.CheckTessellationInput(buildings)

Generate tessellation limited to 100 m buffer. Beware, it is memory demanding.

In [ ]:
limit = mm.buffered_limit(buildings, 100)

tess = mm.Tessellation(buildings, "uID", limit)
tessellation = tess.tessellation

We save tessellation to file.

In [ ]:
tessellation.to_file("files/geometry.gpkg", layer="tessellation", driver="GPKG")

### Tessellation based blocks

To generate tessellation based blocks we also need street network, therefore we need to read it to GeoDataFrame first.

In [ ]:
streets = gpd.read_file('files/geometry.gpkg', layer='streets')

In [ ]:
snapped = mm.snap_street_network_edge(streets, buildings, 20, tessellation, 120, limit) # snap to close unwanted gaps
blocks = mm.Blocks(tessellation, snapped, buildings, 'bID', 'uID')
blocks_df = blocks.blocks  # get blocks df
buildings['bID'] = blocks.buildings_id  # get block ID
tessellation['bID'] = blocks.tessellation_id  # get block ID

#### Link streets

We need to understand which building belongs to which street segment. We link IDs together based on proximity.

In [ ]:
streets["nID"] = range(len(streets))
buildings['nID'] = mm.get_network_id(buildings, streets, 'nID', min_size=300)  # 
tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

Finally, we save elements to a file.

In [ ]:
path = 'files/geometry.gpkg'
tessellation.to_file(path, layer='tessellation', driver='GPKG')
buildings.to_file(path, layer='buildings', driver='GPKG')
blocks_df.to_file(path, layer='blocks', driver='GPKG')

## Measure primary characters

This part measures 74 primary morphometric characters.

It does save intermediate parquet files as a backup.

In [ ]:
tess = tessellation
blg = buildings
blocks = blocks_df

Note: 

- `sdbHei` is building height in meters. If you do not have it, skip affected lines.
- `floor_area` is gross floor area (area * number of floors). If you do not have it, skip affected lines.

In [ ]:
blg['sdbAre'] = mm.Area(blg).series
blg['sdbVol'] = mm.Volume(blg, 'sdbHei', 'sdbAre').series
blg['sdbPer'] = mm.Perimeter(blg).series
blg['sdbCoA'] = mm.CourtyardArea(blg, 'sdbAre').series

blg['ssbFoF'] = mm.FormFactor(blg, 'sdbVol', 'sdbAre').series
blg['ssbVFR'] = mm.VolumeFacadeRatio(blg, 'sdbHei', 'sdbVol', 'sdbPer').series
blg['ssbCCo'] = mm.CircularCompactness(blg, 'sdbAre').series
blg['ssbCor'] = mm.Corners(blg).series
blg['ssbSqu'] = mm.Squareness(blg).series
blg['ssbERI'] = mm.EquivalentRectangularIndex(blg, 'sdbAre', 'sdbPer').series
blg['ssbElo'] = mm.Elongation(blg).series

In [ ]:
cencon = mm.CentroidCorners(blg)
blg['ssbCCM'] = cencon.mean
blg['ssbCCD'] = cencon.std

In [ ]:
blg['stbOri'] = mm.Orientation(blg).series
 
tess['stcOri'] = mm.Orientation(tess).series
blg['stbCeA'] = mm.CellAlignment(blg, tess, 'stbOri', 'stcOri', 'uID', 'uID').series

In [ ]:
tess['sdcLAL'] = mm.LongestAxisLength(tess).series
tess['sdcAre'] = mm.Area(tess).series
tess['sscCCo'] = mm.CircularCompactness(tess, 'sdcAre').series
tess['sscERI'] = mm.EquivalentRectangularIndex(tess, 'sdcAre').series

In [ ]:
blg["mtbSWR"] = mm.SharedWallsRatio(blg, "uID", "sdbPer").series
 
queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tess, ids="uID")
 
blg["mtbAli"] = mm.Alignment(blg, queen_1, "uID", "stbOri").series
blg["mtbNDi"] = mm.NeighborDistance(blg, queen_1, "uID").series
tess["mtcWNe"] = mm.Neighbors(tess, queen_1, "uID", weighted=True).series
tess["mdcAre"] = mm.CoveredArea(tess, queen_1, "uID").series

In [ ]:
blg_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blg, silence_warnings=True)
 
blg["libNCo"] = mm.Courtyards(blg, spatial_weights=blg_q1).series
blg["ldbPWL"] = mm.PerimeterWall(blg, blg_q1).series
 
blocks["ldkAre"] = mm.Area(blocks).series
blocks["ldkPer"] = mm.Perimeter(blocks).series
blocks["lskCCo"] = mm.CircularCompactness(blocks, "ldkAre").series
blocks["lskERI"] = mm.EquivalentRectangularIndex(blocks, "ldkAre", "ldkPer").series
blocks["lskCWA"] = mm.CompactnessWeightedAxis(blocks, "ldkAre", "ldkPer").series
blocks["ltkOri"] = mm.Orientation(blocks).series
 
blo_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blocks, ids="bID")
 
blocks["ltkWNB"] = mm.Neighbors(blocks, blo_q1, "bID", weighted=True).series
blocks["likWBB"] = mm.Count(blocks, blg, "bID", "bID", weighted=True).series

Save data to parquets.

In [ ]:
tess.drop(columns='geometry').to_parquet('files/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('files/blg_data.parquet')
blocks.drop(columns='geometry').to_parquet('files/blocks_data.parquet')

In [ ]:
queen3 = mm.sw_high(k=3, weights=queen_1)
queen1 = queen_1
blg_queen = blg_q1

blg['ltbIBD'] = mm.MeanInterbuildingDistance(blg, queen1, 'uID', queen3).series
blg['ltcBuA'] = mm.BuildingAdjacency(blg, queen3, 'uID', blg_queen).series

In [ ]:
tess = tess.merge(blg[['floor_area', 'uID']], on='uID', how='left')
tess['licGDe'] = mm.Density(tess, 'floor_area', queen3, 'uID', 'sdcAre').series
tess = tess.drop(columns='floor_area')
tess['ltcWRB'] = mm.BlocksCount(tess, 'bID', queen3, 'uID').series
tess['sicCAR'] = mm.AreaRatio(tess, blg, 'sdcAre', 'sdbAre', 'uID').series
tess['sicFAR'] = mm.AreaRatio(tess, blg, 'sdcAre', 'floor_area', 'uID').series

Save data to parquets and spatial weights matrices to gal files.

In [ ]:
tess.drop(columns='geometry').to_parquet('files/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('files/blg_data.parquet')
 
fo = libpysal.io.open('files/AMSqueen1.gal', 'w')
fo.write(queen1)
fo.close()
 
fo = libpysal.io.open('files/AMSqueen3.gal', 'w')
fo.write(queen3)
fo.close()
 
fo = libpysal.io.open('files/AMSblg_queen.gal', 'w')
fo.write(blg_queen)
fo.close()

In [ ]:
streets["sdsLen"] = mm.Perimeter(streets).series
tess["stcSAl"] = mm.StreetAlignment(tess, streets, "stcOri", "nID").series
blg["stbSAl"] = mm.StreetAlignment(blg, streets, "stbOri", "nID").series

In [ ]:
profile = mm.StreetProfile(streets, blg, heights='sdbHei', distance=3)
streets["sdsSPW"] = profile.w
streets["sdsSPH"] = profile.h
streets["sdsSPR"] = profile.p
streets["sdsSPO"] = profile.o
streets["sdsSWD"] = profile.wd
streets["sdsSHD"] = profile.hd

In [ ]:
streets["sssLin"] = mm.Linearity(streets).series
streets["sdsAre"] = mm.Reached(streets, tess, "nID", "nID", mode="sum", values="sdcAre").series
streets["sisBpM"] = mm.Count(streets, blg, "nID", "nID", weighted=True).series

In [ ]:
tess.drop(columns='geometry').to_parquet('files/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('files/blg_data.parquet')
streets.drop(columns='geometry').to_parquet('files/streets_data.parquet')

In [ ]:
str_q1 = libpysal.weights.contiguity.Queen.from_dataframe(streets)
 
streets["misRea"] = mm.Reached(
    streets, tess, "nID", "nID", spatial_weights=str_q1, mode="count"
).series
streets["mdsAre"] = mm.Reached(streets, tess, "nID", "nID", spatial_weights=str_q1,
                               mode="sum").series

In [ ]:
graph = mm.gdf_to_nx(streets)
 
print("node degree")
graph = mm.node_degree(graph)
 
print("subgraph")
graph = mm.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
)
print("cds length")
graph = mm.cds_length(graph, radius=3, name="ldsCDL")
 
print("clustering")
graph = mm.clustering(graph, name="xcnSCl")
 
print("mean_node_dist")
graph = mm.mean_node_dist(graph, name="mtdMDi")
 
nodes, edges, sw = mm.nx_to_gdf(graph, spatial_weights=True)

In [ ]:
nodes.to_file(path, layer="nodes", driver="GPKG")
edges.to_file(path, layer="edges", driver="GPKG")
 
fo = libpysal.io.open("files/nodes.gal", "w")
fo.write(sw)
fo.close()

In [ ]:
edges_w3 = mm.sw_high(k=3, gdf=edges)
edges["ldsMSL"] = mm.SegmentsLength(edges, spatial_weights=edges_w3, mean=True).series
 
edges["ldsRea"] = mm.Reached(edges, tess, "nID", "nID", spatial_weights=edges_w3).series
edges["ldsRea"] = mm.Reached(
    edges, tess, "nID", "nID", spatial_weights=edges_w3, mode="sum", values="sdcAre"
).series
 
nodes_w5 = mm.sw_high(k=5, weights=sw)
nodes["lddNDe"] = mm.NodeDensity(nodes, edges, nodes_w5).series
nodes["linWID"] = mm.NodeDensity(
    nodes, edges, nodes_w5, weighted=True, node_degree="degree"
).series
 
blg["nodeID"] = mm.get_node_id(blg, nodes, edges, "nodeID", "nID")
tess = tess.merge(blg[["uID", "nodeID"]], on="uID", how="left")
 
nodes_w3 = mm.sw_high(k=3, weights=sw)
 
nodes["lddRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", nodes_w3).series
nodes["lddARe"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", nodes_w3, mode="sum", values="sdcAre"
).series
 
nodes["sddAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", mode="sum", values="sdcAre"
).series
nodes["midRea"] = mm.Reached(nodes, tess, "nodeID", "nodeID", spatial_weights=sw).series
nodes["midAre"] = mm.Reached(
    nodes, tess, "nodeID", "nodeID", spatial_weights=sw, mode="sum", values="sdcAre"
).series
 
nodes.rename(
    columns={
        "degree": "mtdDeg",
        "meshedness": "lcdMes",
        "local_closeness": "lcnClo",
        "proportion_3": "linP3W",
        "proportion_4": "linP4W",
        "proportion_0": "linPDE",
    }, inplace=True
)

In [ ]:
tess.drop(columns='geometry').to_parquet('files/tess_data.parquet')
blg.drop(columns='geometry').to_parquet('files/blg_data.parquet')
nodes.drop(columns='geometry').to_parquet('files/nodes_data.parquet')
edges.drop(columns='geometry').to_parquet('files/edges_data.parquet')

In [ ]:
merged = tess.merge(blg.drop(columns=['nID', 'bID', 'nodeID', 'geometry']), on='uID')
merged = merged.merge(blocks.drop(columns='geometry'), on='bID', how='left')
merged = merged.merge(edges.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')

Clean columns to keep only measured data.

In [ ]:
primary = merged.drop(columns=['nID', 'bID', 'nodeID', 'mm_len', 'cdsbool', 
                               'node_start', 'node_end', 'geometry', 'floor_area'
                               ])

In [ ]:
primary.to_parquet('files/primary.parquet')

## Measure contextual - spatially lagged characters

This part measures contextual characters.

In [ ]:
gdf = primary.set_index('uID')
spatial_weights = queen3
unique_id = 'uID'

In [ ]:
means = {}
ranges = {}
theils = {}

for ch in gdf.columns:
    means[ch] = []
    ranges[ch] = []
    theils[ch] = []

Resolve potential missingness cause by invalid input data. That was not case in the presented case studies but may be case in subsequent research.

In [ ]:
gdf = gdf.replace(np.inf, np.nan)  # normally does not happen, but to be sure
gdf = gdf.fillna(0)  # normally does not happen, but to be sure
chars = gdf.columns

In [ ]:
gdf['lcdMes'] = gdf.apply(
            lambda row: row.lcdMes if row.lcdMes >= 0 else 0,
            axis=1,
        )  # normally does not happen, but to be sure

Define Theil and Simpson functions.

In [ ]:
def theil(y):
    y = np.array(y)
    n = len(y)
    plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
    yt = plus.sum(axis=0)
    s = plus / (yt * 1.0)
    lns = np.log(n * s)
    slns = s * lns
    t = sum(slns)
    return t

In [ ]:
def _simpson_di(data):

    def p(n, N):
        if n == 0:
            return 0
        return float(n) / N

    N = sum(data.values())

    return sum(p(n, N) ** 2 for n in data.values() if n != 0)

Loop over DataFrame and measure IQM, IQR and IDT.

In [ ]:
for index in tqdm(range(len(gdf)), total=gdf.shape[0]):
    neighbours = [index]
    neighbours += spatial_weights.neighbors[index]
    
    subset = gdf.iloc[neighbours]
    for ch in chars:
        values_list = subset[ch] 
        idec = limit_range(values_list, rng=(10, 90))
        iquar = limit_range(values_list, rng=(25, 75))
        
        means[ch].append(np.mean(iquar))
        ranges[ch].append(max(iquar) - min(iquar))
        theils[ch].append(theil(idec))

In [ ]:
contextual = pd.DataFrame(index=gdf.index)
for ch in chars:
    contextual[ch + '_meanIQ3'] = means[ch]
    contextual[ch + '_rangeIQ3'] = ranges[ch]
    contextual[ch + '_theilID3'] = theils[ch]

Now we measure Simpson's diversity.

Skewness is used as an estimation of the distribution. Extremely skewed use HeadTail breaks for Simpson's binning, other Natural Breaks.

In [ ]:
skewness = pd.DataFrame(index=chars)
for c in chars:
    skewness.loc[c, 'skewness'] = sp.stats.skew(gdf[c])
headtail = list(skewness.loc[skewness.skewness >= 1].index)
to_invert = list(skewness.loc[skewness.skewness <= -1].index)

for inv in to_invert:
    gdf[inv] = gdf[inv].max() - gdf[inv]
headtail = headtail + to_invert
natural = [c for c in chars if c not in headtail]

In [ ]:
len(natural) + len(headtail)

We create global bins.

In [ ]:
results = {}
for c in headtail + natural:
    results[c] = []
bins = {}
for c in headtail:
    bins[c] = mapclassify.HeadTailBreaks(gdf[c]).bins
for c in natural:
    bins[c] = mapclassify.gadf(gdf[c], method='NaturalBreaks')[1].bins

And measure local Simpson's diversity.

In [ ]:
for index in tqdm(gdf.index, total=gdf.shape[0]):
    neighbours = [index]
    neighbours += spatial_weights.neighbors[index]
    
    subset = gdf.loc[neighbours]
    for c in headtail + natural:
        values = subset[c]
        sample_bins = mapclassify.UserDefined(values, list(bins[c]))
        counts = dict(zip(bins[c], sample_bins.counts))
        results[c].append(_simpson_di(counts))

In [ ]:
for c in headtail + natural:
    contextual[c + '_simpson'] = results[c]

In [ ]:
contextual.shape

In [ ]:
contextual.to_parquet('files/contextual.parquet')

## Clustering

We use contextual characters to do GMM clustering.

In [ ]:
data = contextual

In [ ]:
data

First we standardize data.

In [ ]:
# normalise data

x = data.values
scaler = preprocessing.StandardScaler()
cols = list(data.columns)
data[cols] = scaler.fit_transform(data[cols])

Measure BIC to estimate optimal number of clusters.

In [ ]:
bic = pd.DataFrame(columns=['n', 'bic', 'run'])
ix = 0

n_components_range = range(2, 6) # specify range you want to assess
gmmruns = 3  # specify how many times should each option be tried (more better, but takes a long time)

In [ ]:
data = data.fillna(0)
for n_components in n_components_range:
    for i in range(gmmruns):
        gmm = GaussianMixture(n_components=n_components, covariance_type="full", max_iter=200, n_init=1, verbose=1)
        fitted = gmm.fit(data)
        bicnum = gmm.bic(data)
        bic.loc[ix] = [n_components, bicnum, i]
        ix += 1

        print(n_components, i, "BIC:", bicnum)

In [ ]:
bic.to_csv('files/complete_BIC.csv')

Based on the plot below, we estimate the optimal `n` as the first significant minimum.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
sns.lineplot(ax=ax, x='n', y='bic', data=bic)
# plt.savefig('files/complete_BIC.pdf')

### Gaussian Mixture Model

In [ ]:
n = 4  # based on above
n_init = 5  # more initialization, more stable clustering gets

gmm = GaussianMixture(n_components=n, covariance_type="full", max_iter=200, n_init=n_init, verbose=1)
fitted = gmm.fit(data)

In [ ]:
data['cluster'] = gmm.predict(data)

In [ ]:
data.reset_index()[['cluster', 'uID']].to_csv('files/cluster_labels.csv')

In [ ]:
clusters = data.reset_index()[['cluster', 'uID']]

#### Hierachical clustering

Finally, we create hierarchical classification - taxonomy.

In [ ]:
group = data.groupby('cluster').mean()
Z = hierarchy.linkage(group, 'ward')
plt.figure(figsize=(25, 10))
dn = hierarchy.dendrogram(Z, labels=group.index)

# plt.savefig('tree.pdf')